## Calculate Phi spatial average within a basin or within LMEs

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
#tres = 'monthly'
tres = 'yearly'

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
darea = darea*dz/dz
darea = darea.transpose("z_t","nlat","nlon")

In [5]:
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            darea[kk:, xx, yy] = float("nan")

In [6]:
filename = f'{path}/DPLE_driftcorrected_Phi_Eo_space_ens_mean_{tres}.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:  (Y: 64, year: 10, z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * nlat     (nlat) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * nlon     (nlon) int64 0 1 2 3 4 5 6 7 8 ... 187 188 189 190 191 192 193 194
Data variables: (12/13)
    Phi_0    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_1    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_2    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_3    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_4    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_5    (Y, year, z_t, nlat, nlon) float64 ...
    ...       ...
    Phi_7    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_8    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_9    (Y, year, z_t, nlat, nlon) float64 ...
    Phi_10   (Y, year, z_t, nlat, nlon) float64 ...
    Phi_11   (Y, year, z_t, nlat, nlon) float64 ...
    Phi_12   (Y, year, z_t, nlat, nlon) float64 ...

In [7]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
lme = [1,2,3,4,5,6,7,8,9,10,65]

In [8]:
if tres == 'yearly':
    tnam = 'year'    
else:
    tnam = 'L'
nlead = ds.dims[tnam]
nyear = ds.dims['Y']
nver  = ds.dims['z_t']

In [9]:
%%time
dsf = xr.Dataset()
for ii in range(13):
    print(ii)
    dsf[f'Phi_{ii}'] = ds[f'Phi_{ii}']*darea

0
1
2
3
4
5
6
7
8
9
10
11
12
CPU times: user 35.2 s, sys: 29.8 s, total: 1min 5s
Wall time: 1min 9s


In [10]:
%%time
phi_box = np.ma.zeros([13, len(lme), nyear, nlead, nver])
phi_box.mask = True
# select a LME
for box,nn in zip(lme,range(len(lme))):
    print(box)
    latloc, lonloc = np.where(lme_mask == box)
    ar1 = np.ma.zeros([len(latloc),nver]);ar1.mask=True
    ar2 = np.ma.zeros([13, len(latloc),nyear,nlead,nver]);ar2.mask=True
    for bb in range(len(latloc)):
        ar1[bb] = darea.isel(nlat=latloc[bb], nlon=lonloc[bb])
        for ii in range(13):
            ar2[ii,bb] = dsf[f'Phi_{ii}'].isel(nlat=latloc[bb], nlon=lonloc[bb])
    #
    tarea = np.nansum(ar1,axis=0)
    tarea = np.tile(tarea[np.newaxis,np.newaxis,np.newaxis,:],[13,nyear,nlead,1])
    phi_box[:,nn] = np.nansum(ar2,axis=1)/tarea   

1
2
3
4
5
6
7
8
9
10
65
CPU times: user 37 s, sys: 2.06 s, total: 39.1 s
Wall time: 41.1 s


In [11]:
dsf = xr.Dataset()
#
for ii in range(13):
    dsf[f'Phi_{ii}'] = xr.DataArray(data=phi_box[ii], coords=dict(lme=('lme', lme), Y=('Y', ds.coords['Y'].values), year=('year', ds.coords[tnam].values), z_t=('z_t', ds.coords['z_t'].values)), dims=('lme','Y','year','z_t'))

In [12]:
dsf.load()

<xarray.Dataset>
Dimensions:  (lme: 11, Y: 64, year: 10, z_t: 35)
Coordinates:
  * lme      (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
Data variables: (12/13)
    Phi_0    (lme, Y, year, z_t) float64 1.055 1.056 1.059 ... 0.07706 0.07288
    Phi_1    (lme, Y, year, z_t) float64 1.225 1.226 1.229 ... 0.08949 0.08475
    Phi_2    (lme, Y, year, z_t) float64 1.421 1.423 1.427 ... 0.1039 0.09857
    Phi_3    (lme, Y, year, z_t) float64 1.65 1.652 1.656 ... 0.1207 0.1146
    Phi_4    (lme, Y, year, z_t) float64 1.916 1.918 1.923 ... 0.1402 0.1333
    Phi_5    (lme, Y, year, z_t) float64 2.225 2.228 2.234 ... 0.1628 0.155
    ...       ...
    Phi_7    (lme, Y, year, z_t) float64 3.003 3.007 3.016 ... 0.2195 0.2097
    Phi_8    (lme, Y, year, z_t) float64 3.489 3.494 3.505 ... 0.255 0.2439
    Phi_9    (lme, Y, year, z_t) float64 4.056 4.061 4.075 ... 0.2961 0.2836
    Phi_10   (lme, Y, year, z_t) float64 4.715 4.721 4.738 ... 0.3439 0.3299
    Phi_11   (lme, Y, year, z_t) float64 5.482 5.49 5.51 ... 0.3994 0.3837
    Phi_12   (lme, Y, year, z_t) float64 6.376 6.385 6.409 ... 0.4638 0.4462

In [13]:
dsf.to_netcdf(f'{path}/DPLE_driftcorrected_Phi_Eo_space_horizontal_mean_LME_ens_mean_{tres}.nc', mode='w')